In [1]:
import selenium
from selenium import webdriver
import os
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
print('-*-'*50)
print('2021-2 Marketing analytics')
print('디시인사이드 갤럭시 커뮤니티 게시글+댓글 크롤러')
print('-*-'*50)

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'
os.makedirs(f_dir+s+'-'+'디시_갤럭시_커뮤니티_게시글과_댓글')
os.chdir(f_dir+s+'-'+'디시_갤럭시_커뮤니티_게시글과_댓글')
cnt = int(input('몇 건 게시물을 수집하시겠습니까?'))

driver_path = '/Users/kibeomkim/Desktop/chromedriver_intel/m1_chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')
driver = webdriver.Chrome(driver_path, options=options)
driver.get('https://gall.dcinside.com/mgallery/board/lists?id=galaxy')
s_time = time.time()
time.sleep(random.randint(4,8))

-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
2021-2 Marketing analytics
디시인사이드 갤럭시 커뮤니티 게시글+댓글 크롤러
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-


In [3]:
# part 1: 페이지 파싱해서 url 주소만 전부 수집. 리스트에 추가하기 
page_no = 1
url_list = []
no = 0

while True: # 페이지 전환
    loaded_page = driver.page_source
    soup = BeautifulSoup(loaded_page, 'html.parser')
    tr_list = soup.find_all('tr', 'ub-content us-post')
    
    for tr in tr_list : 
        writing_type = tr.find('td', 'gall_subject').get_text()
        if writing_type == '공지' :  # 공지글인지 아닌지 검사하기 
            continue # 공지글이면 패스 
        else : 
            url = tr.find('a')['href']
            no += 1
            if no > cnt : 
                break
            url_list.append(url)
    if no > cnt : 
        break
    page_no += 1 # 다음 페이지 번호 

    for scroll in range(4) : # 스크롤 다운
        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        time.sleep(4)

    try : 
        time.sleep(3)
        driver.find_element_by_link_text(str(page_no)).click()
        time.sleep(random.randint(5,8))
    except : 
        try : 
            time.sleep(3)
            driver.find_element_by_link_text('다음').click() # 다음 페이지로 넘어가기 
            time.sleep(5)

            driver.find_element_by_link_text(str(page_no)).click() # 원래 넘어가야 하는 페이지 번호 다시 클릭하기 
            time.sleep(random.randint(5,9))
        except : 
            print('다음 페이지가 없습니다')
            break
driver.quit()
time.sleep(2)

In [ ]:
# part 2 : 페이지 순차 접속 후 게시 정보, 본문 내용, 그리고 댓글 정보 수집하기 
title_list = []
writer_list = []
posting_date_list = []
contents_list = []

for url in url_list : 
    # 크롬 드라이버 호출
    options = webdriver.ChromeOptions()
    options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')
    driver = webdriver.Chrome(driver_path, options=options)

    # 페이지 접속 
    driver.get('https://gall.dcinside.com'+url)
    time.sleep(random.randint(5,9)) # 페이지 로드 대기 

    # 페이지 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 게시글 제목 수집 
    title = soup.find('span', 'title_subject').get_text().replace(' ', "")
    title_list.append(title)
    print(f'게시글 제목 : {title}')

    # 게시자 수집 
    writer = soup.find('span', 'nickname in').get_text().replace(' ', "")
    writer_list.append(writer)
    print(f'게시자 : {writer}')

    # 게시 일자 수집 
    posting_date = soup.find('span', 'gall_date').get_text().replace(' ', "")
    posting_date_list.append(posting_date)
    print(f'게시 일자 : {posting_date}')

    # 본문 내용 수집 
    contents = soup.find('div', 'write_div').get_text().replace('\n', "")
    contents_list.append(contents)
    print(f'본문 내용 : {contents}')


For 문으로 url_list 순환 : 
1. 드라이버 호출 
2. url로 접속 후 3초 대기
3. 페이지 파싱

1. 게시글 제목 수집
2. 게시자 수집
3. 게시 일자 수집 
4. Class = write_div 로 div 태그 찾고, 본문 내용 수집하기 


1. try: 댓글 태그 있는지 확인
- 댓글 태그 있으면: 댓글 게시자, 게시일자, 댓글 내용 수집 
- driver.quit() 로 해당 페이지 크롤링 종료 

2. Except : 
- (댓글 태그 없음) 
- driver.quit() 로 해당 페이지 크롤링 종료 